### SVM linear

### 載入套件

In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from time import time
from sklearn import metrics
import scikitplot as skplt
import matplotlib.pyplot as plt

### panda 語法

In [ ]:
## df合併
pd.concat([df_embedding, df_cvaw_covid], axis = 1)    # axis = 1：橫向合併，axis = 0：直向合併
## 取列(row)
df_embedding[0:2]    
## 取指定名字的col
df_embedding[["0","3"]]    
## 取指定欄數的col
df_embedding.iloc[:, 1:3]    

### 讀檔

情緒詞與 y 不會因為斷詞方式不同而有改變

In [6]:
## cvaw + stock sign
filePath = "./preprocess/tokens+stockSign/"
df_cvaw_covid = pd.read_csv(filePath + "cvaw_covid_stockSign.csv")[["Valence_Sum","Arousal_Sum", "words_Num"]]
df_cvaw_stock = pd.read_csv(filePath + "cvaw_stock_stockSign.csv")[["Valence_Sum","Arousal_Sum", "words_Num"]]
df_stockSign_covid = pd.read_csv(filePath + "cvaw_covid_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]
df_stockSign_stock = pd.read_csv(filePath + "cvaw_stock_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]

In [55]:
df_cvaw_covid

,Valence_Sum,Arousal_Sum,words_Num
0,-16.6,-18.8,34
1,23.3,-6.2,69
2,-76.0,-50.7,125
3,11.2,-10.0,29
4,-13.3,-21.2,41
...,...,...,...
7509,2.0,-7.6,8
7510,-32.8,-0.8,47
7511,0.6,-1.4,4
7512,-10.7,-18.2,29


### 函式：讀 tokens data 並併入情緒詞，切成 x_train, x_test

In [3]:
def SVD_embedding(tokens_name, df_cvaw):
    df_embedding = pd.read_csv("./TFIDF/" + tokens_name + "_stockSign_TFIDF_TruncatedSVD.csv")
    
    df_embedding_cvaw = pd.concat([df_embedding, df_cvaw], axis = 1).iloc[:, 1:504]
    
    x_train, x_test = train_test_split(df_embedding_cvaw, test_size=0.20, random_state=404)
    
    return x_train, x_test

In [4]:
def word2vec_embedding(tokens_name, df_cvaw):
    df_embedding = pd.read_csv("./Word2Vec/" + tokens_name + "_stockSign_word2vec.csv")
    
    df_embedding_cvaw = pd.concat([df_embedding, df_cvaw], axis = 1).iloc[:, 1:504]
    
    x_train, x_test = train_test_split(df_embedding_cvaw, test_size=0.20, random_state=404)
    
    return x_train, x_test

## covid 板

#### covid 板 y label

In [11]:
y_train, y_test = train_test_split(df_stockSign_covid, test_size=0.20, random_state=404)

In [12]:
y_train["stockRise_mask"].value_counts()

 1    2374
-1    2305
 0    1332
Name: stockRise_mask, dtype: int64

In [13]:
y_train["stockRise_testKits"].value_counts()

 0    2501
 1    1807
-1    1703
Name: stockRise_testKits, dtype: int64

In [14]:
y_train["stockRise_vaccine"].value_counts()

 0    3197
 1    1481
-1    1333
Name: stockRise_vaccine, dtype: int64

In [9]:
y_train

,date,stockRise_mask,stockRise_testKits,stockRise_vaccine
7271,2021-05-20,0,0,1
317,2020-02-11,-1,0,0
1925,2020-03-17,1,0,0
3188,2020-04-28,1,1,-1
7289,2021-05-23,-1,1,1
...,...,...,...,...
6031,2021-01-18,-1,1,1
5108,2020-10-28,1,0,0
5302,2020-11-19,0,-1,0
5994,2021-01-14,0,0,-1


In [10]:
y_train["stockRise_mask"]

7271    0
317    -1
1925    1
3188    1
7289   -1
       ..
6031   -1
5108    1
5302    0
5994    0
5884    1
Name: stockRise_mask, Length: 6011, dtype: int64

In [11]:
y_train_mask = y_train["stockRise_mask"].tolist()    # 單括號取list，雙括號取dataframe
y_train_testKits = y_train["stockRise_testKits"].tolist()
y_train_vaccine = y_train["stockRise_vaccine"].tolist()
y_test_mask = y_test["stockRise_mask"].tolist()
y_test_testKits = y_test["stockRise_testKits"].tolist()
y_test_vaccine = y_test["stockRise_vaccine"].tolist()

### covid 板 jieba 斷詞 testing & training split

In [71]:
x_train_covid_token, x_test_covid_token = SVD_embedding("covid_token", df_cvaw_covid)

In [72]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_token.index)):
    if x_train_covid_token.index[i] != y_train.index[i]:
        print(x_train_covid_token.index[i])

In [73]:
x_train_covid_token

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
7271,0.168230,-0.069868,0.077066,-0.034568,0.057927,0.000642,0.073645,-0.033365,0.041456,-0.023926,...,-0.042941,-0.019400,-0.001544,0.001212,-0.004110,-0.010077,0.000500,-8.4,-5.6,8
317,0.232638,-0.070729,0.081800,-0.080535,0.052000,-0.021252,0.083795,0.027894,0.027037,-0.014778,...,0.026503,-0.016544,-0.013913,0.004980,-0.027854,-0.004037,0.029034,-4.4,-13.2,28
1925,0.078999,0.015901,-0.035217,-0.024267,-0.010944,-0.010970,-0.004552,0.028118,0.014095,-0.006812,...,-0.012236,0.042490,-0.009440,0.007456,-0.018702,-0.006273,0.012803,-0.6,-3.4,3
3188,0.123637,0.020223,-0.019573,-0.197853,0.257071,-0.017419,-0.222864,-0.194154,-0.097828,0.043375,...,0.027337,-0.009452,0.014707,0.008425,0.027192,0.015493,0.012912,11.8,-10.2,17
7289,0.078144,0.075450,0.021081,0.000825,-0.018169,0.040393,0.008424,-0.034574,0.024557,-0.014674,...,0.007203,-0.029034,-0.003422,0.015963,-0.000014,-0.006183,0.003564,-17.8,-2.6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.447681,-0.322748,0.404306,0.018911,-0.205223,0.073825,-0.029511,-0.098586,-0.013720,0.028318,...,0.000623,-0.003180,-0.000004,0.024793,-0.018361,0.006151,0.003587,-1.2,-40.0,56
5108,0.117317,0.031793,-0.044411,-0.105747,0.070221,0.006170,0.001168,0.032819,-0.057208,0.031430,...,-0.015356,0.009724,-0.013975,0.001282,-0.028884,0.000371,-0.000259,-5.8,-3.0,54
5302,0.169029,0.286873,0.054608,0.062615,-0.064380,-0.019316,-0.028392,0.004769,-0.026208,0.019041,...,-0.008264,0.013812,-0.018826,-0.005285,0.012279,0.004206,-0.015715,8.6,-5.5,13
5994,0.149840,-0.074199,0.124232,-0.079804,-0.027807,0.033769,-0.045400,0.058291,0.015603,-0.004764,...,0.009267,-0.000793,-0.005890,0.017886,0.017764,-0.029175,0.002483,-9.3,-5.1,12


In [74]:
x_train_covid_token = x_train_covid_token.values.tolist()
x_test_covid_token = x_test_covid_token.values.tolist()

### covid 板 ckip 斷詞 testing & training split

In [36]:
x_train_covid_CKIPtoken, x_test_covid_CKIPtoken = SVD_embedding("covid_CKIPtoken", df_cvaw_covid)

In [37]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_CKIPtoken.index)):
    if x_train_covid_CKIPtoken.index[i] != y_train.index[i]:
        print(x_train_covid_CKIPtoken.index[i])

In [38]:
x_train_covid_CKIPtoken = x_train_covid_CKIPtoken.values.tolist()
x_test_covid_CKIPtoken = x_test_covid_CKIPtoken.values.tolist()

### covid 板 ckip nonEng 斷詞 testing & training split

In [12]:
x_train_covid_CKIP_nonEng, x_test_covid_CKIP_nonEng = SVD_embedding("covid_CKIP_nonEng", df_cvaw_covid)

In [13]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_CKIP_nonEng.index)):
    if x_train_covid_CKIP_nonEng.index[i] != y_train.index[i]:
        print(x_train_covid_CKIP_nonEng.index[i])

In [14]:
x_train_covid_CKIP_nonEng = x_train_covid_CKIP_nonEng.values.tolist()
x_test_covid_CKIP_nonEng = x_test_covid_CKIP_nonEng.values.tolist()

## stock 板

#### stock 板 y label

In [7]:
y_train, y_test = train_test_split(df_stockSign_stock, test_size=0.20, random_state=404)

In [8]:
y_train["stockRise_mask"].value_counts()

 1    3102
-1    2809
 0    1872
Name: stockRise_mask, dtype: int64

In [9]:
y_train["stockRise_testKits"].value_counts()

 0    3277
 1    2310
-1    2196
Name: stockRise_testKits, dtype: int64

In [10]:
y_train["stockRise_vaccine"].value_counts()

 0    4238
-1    1810
 1    1735
Name: stockRise_vaccine, dtype: int64

In [33]:
y_train

,date,stockRise_mask,stockRise_testKits,stockRise_vaccine
6004,2020-11-15,0,0,0
6702,2021-01-05,1,1,-1
4971,2020-08-12,0,-1,0
1867,2020-03-23,0,-1,0
2484,2020-04-09,0,-1,0
...,...,...,...,...
6031,2020-11-16,1,0,0
5108,2020-08-20,1,-1,0
5302,2020-09-07,0,0,1
5884,2020-11-08,1,1,0


In [34]:
y_train["stockRise_mask"]

6004    0
6702    1
4971    0
1867    0
2484    0
       ..
6031    1
5108    1
5302    0
5884    1
7911   -1
Name: stockRise_mask, Length: 7783, dtype: int64

In [35]:
y_train_mask = y_train["stockRise_mask"].tolist()    # 單括號取list，雙括號取dataframe
y_train_testKits = y_train["stockRise_testKits"].tolist()
y_train_vaccine = y_train["stockRise_vaccine"].tolist()
y_test_mask = y_test["stockRise_mask"].tolist()
y_test_testKits = y_test["stockRise_testKits"].tolist()
y_test_vaccine = y_test["stockRise_vaccine"].tolist()

### stock 板 jieba 斷詞 testing & training split

In [43]:
x_train_stock_token, x_test_stock_token = SVD_embedding("stock_token", df_cvaw_stock)

In [44]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_token.index)):
    if x_train_stock_token.index[i] != y_train.index[i]:
        print(x_train_stock_token.index[i])

In [45]:
x_train_stock_token

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
6004,0.110194,-0.056617,0.016188,0.051068,0.024915,-0.024274,-0.014723,-0.026258,-0.055791,-0.005208,...,0.003994,0.017639,-0.015786,0.002318,0.026095,0.000526,0.005658,3.0,-2.6,12
6702,0.215222,-0.029807,-0.106440,-0.060623,-0.081690,-0.008828,-0.005497,0.125946,-0.049947,-0.023866,...,-0.016545,-0.014984,0.025858,-0.018918,-0.001705,0.016183,0.027335,35.6,-22.2,48
4971,0.190443,0.456539,-0.089589,0.113550,0.085209,-0.011482,-0.013740,0.040625,-0.076117,0.018417,...,-0.018082,0.016909,0.012102,0.003540,-0.022592,0.000633,-0.007836,14.0,-12.4,29
1867,0.230840,-0.021076,-0.092870,-0.061819,-0.110322,-0.008187,0.002349,0.030462,0.027457,0.002368,...,0.010719,-0.002002,0.002424,0.010414,-0.025819,0.006383,0.013708,8.2,-33.6,39
2484,0.153676,-0.052038,0.060150,0.017483,0.192235,0.037284,-0.009671,0.023961,0.167582,-0.376188,...,0.012238,-0.005709,0.024119,-0.035278,0.012393,-0.016911,0.011092,0.1,-15.9,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.114674,0.005346,-0.026531,-0.063207,0.020567,0.034371,0.034781,-0.019504,-0.047736,0.029909,...,-0.007536,0.015456,0.012212,0.022440,0.019762,-0.006434,-0.001645,-5.5,6.6,18
5108,0.140593,-0.014271,-0.065399,-0.031681,-0.076528,-0.009261,0.001531,0.053130,-0.032995,-0.036033,...,0.045533,0.011390,0.017935,0.011710,0.002624,-0.019156,-0.023624,50.5,-27.6,57
5302,0.080773,-0.011286,0.018688,0.016437,-0.017477,-0.009216,-0.000077,-0.032430,0.000492,0.047174,...,0.004887,-0.012008,0.003088,0.024134,0.012363,-0.005537,0.018533,3.6,-5.8,13
5884,0.158278,0.128662,-0.078809,0.024062,0.000665,-0.018054,0.009116,0.010543,0.001331,-0.067082,...,0.008776,-0.010579,0.002256,-0.008569,0.012491,-0.017333,-0.002944,12.6,-25.0,39


In [46]:
x_train_stock_token = x_train_stock_token.values.tolist()
x_test_stock_token = x_test_stock_token.values.tolist()

### stock 板 ckip 斷詞 testing & training split

In [51]:
x_train_stock_CKIPtoken, x_test_stock_CKIPtoken = SVD_embedding("stock_CKIPtoken", df_cvaw_stock)

In [52]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_CKIPtoken.index)):
    if x_train_stock_CKIPtoken.index[i] != y_train.index[i]:
        print(x_train_stock_CKIPtoken.index[i])

In [53]:
x_train_stock_CKIPtoken = x_train_stock_CKIPtoken.values.tolist()
x_test_stock_CKIPtoken = x_test_stock_CKIPtoken.values.tolist()

### stock 板 ckip nonEng 斷詞 testing & training split

In [61]:
x_train_stock_CKIP_nonEng, x_test_stock_CKIP_nonEng = SVD_embedding("stock_CKIP_nonEng", df_cvaw_stock)

In [62]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_CKIP_nonEng.index)):
    if x_train_stock_CKIP_nonEng.index[i] != y_train.index[i]:
        print(x_train_stock_CKIP_nonEng.index[i])

In [63]:
x_train_stock_CKIP_nonEng = x_train_stock_CKIP_nonEng.values.tolist()
x_test_stock_CKIP_nonEng = x_test_stock_CKIP_nonEng.values.tolist()

## word2vec

## covid 板

### covid 板 jieba 斷詞 testing & training split

In [86]:
x_train_covid_token, x_test_covid_token = word2vec_embedding("covid_token", df_cvaw_covid)

In [87]:
x_train_covid_token

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
7271,0.466977,0.490552,-0.046918,-0.202188,0.166707,0.052072,0.100822,0.127350,0.293276,0.144377,...,0.080226,-0.125111,-0.088842,0.191329,-0.090425,0.034194,-0.061873,-8.4,-5.6,8
317,-0.137972,0.238994,-0.103087,-0.276188,-0.096632,0.055066,-0.054771,0.103535,-0.027499,0.190077,...,-0.107904,-0.122517,-0.083494,0.085071,0.077354,0.072515,0.231514,-4.4,-13.2,28
1925,-0.272390,-0.219359,0.030242,0.378799,0.014876,-0.015882,-0.218432,0.099289,0.055401,0.247769,...,-0.022874,0.055691,-0.276878,0.057967,0.295714,-0.024754,0.151208,-0.6,-3.4,3
3188,-0.165721,0.238822,-0.049381,-0.021363,0.317870,-0.113444,-0.313132,0.128785,0.005647,0.069155,...,0.234333,-0.128627,0.060161,0.052233,-0.132020,0.074416,0.239536,11.8,-10.2,17
7289,-0.231800,-0.203382,0.166360,-0.131830,0.190765,-0.032649,0.228955,-0.392035,-0.171504,0.106874,...,-0.069726,-0.126730,0.170638,0.217938,0.494262,0.491883,0.135065,-17.8,-2.6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.635976,0.370562,-0.194026,-0.273566,0.023036,0.088587,0.273419,0.192486,0.337552,0.173713,...,-0.355655,-0.181885,-0.530296,-0.050131,0.032169,-0.168377,-0.005768,-1.2,-40.0,56
5108,-0.282629,-0.084010,-0.158663,0.005067,-0.008301,-0.057799,-0.283823,-0.172922,-0.028213,0.138294,...,-0.076541,-0.203598,-0.086399,-0.062242,-0.176672,-0.209389,0.280768,-5.8,-3.0,54
5302,-0.183439,-0.147837,-0.012782,0.516806,-0.024616,-0.445040,0.228963,-0.249978,-0.056679,0.074657,...,-0.307480,-0.170445,-0.036629,-0.077072,0.211082,-0.150104,0.504225,8.6,-5.5,13
5994,0.219891,0.571944,0.013836,-0.294966,0.212134,0.060913,0.161956,-0.021638,0.250821,-0.284589,...,0.037604,0.034764,-0.154887,0.087008,-0.282653,-0.279951,-0.046932,-9.3,-5.1,12


In [88]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_token.index)):
    if x_train_covid_token.index[i] != y_train.index[i]:
        print(x_train_covid_token.index[i])

In [89]:
x_train_covid_token = x_train_covid_token.values.tolist()
x_test_covid_token = x_test_covid_token.values.tolist()

### covid 板 ckip 斷詞 testing & training split

In [20]:
x_train_covid_CKIPtoken, x_test_covid_CKIPtoken = word2vec_embedding("covid_CKIPtoken", df_cvaw_covid)

In [21]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_CKIPtoken.index)):
    if x_train_covid_CKIPtoken.index[i] != y_train.index[i]:
        print(x_train_covid_CKIPtoken.index[i])

In [22]:
x_train_covid_CKIPtoken = x_train_covid_CKIPtoken.values.tolist()
x_test_covid_CKIPtoken = x_test_covid_CKIPtoken.values.tolist()

### covid 板 ckip nonEng 斷詞 testing & training split

In [23]:
x_train_covid_CKIP_nonEng, x_test_covid_CKIP_nonEng = word2vec_embedding("covid_CKIP_nonEng", df_cvaw_covid)

In [24]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_covid_CKIP_nonEng.index)):
    if x_train_covid_CKIP_nonEng.index[i] != y_train.index[i]:
        print(x_train_covid_CKIP_nonEng.index[i])

In [25]:
x_train_covid_CKIP_nonEng = x_train_covid_CKIP_nonEng.values.tolist()
x_test_covid_CKIP_nonEng = x_test_covid_CKIP_nonEng.values.tolist()

### stock 板 jieba 斷詞 testing & training split

In [36]:
x_train_stock_token, x_test_stock_token = word2vec_embedding("stock_token", df_cvaw_stock)

In [37]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_token.index)):
    if x_train_stock_token.index[i] != y_train.index[i]:
        print(x_train_stock_token.index[i])

In [38]:
x_train_stock_token = x_train_stock_token.values.tolist()
x_test_stock_token = x_test_stock_token.values.tolist()

### stock 板 ckip 斷詞 testing & training split

In [39]:
x_train_stock_CKIPtoken, x_test_stock_CKIPtoken = word2vec_embedding("stock_CKIPtoken", df_cvaw_stock)

In [40]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_CKIPtoken.index)):
    if x_train_stock_CKIPtoken.index[i] != y_train.index[i]:
        print(x_train_stock_CKIPtoken.index[i])

In [41]:
x_train_stock_CKIPtoken = x_train_stock_CKIPtoken.values.tolist()
x_test_stock_CKIPtoken = x_test_stock_CKIPtoken.values.tolist()

### stock 板 ckip nonEng 斷詞 testing & training split

In [42]:
x_train_stock_CKIP_nonEng, x_test_stock_CKIP_nonEng = word2vec_embedding("stock_CKIP_nonEng", df_cvaw_stock)

In [43]:
# 檢查切出來的 x_train y_train index 是否相同
for i in range(len(x_train_stock_CKIP_nonEng.index)):
    if x_train_stock_CKIP_nonEng.index[i] != y_train.index[i]:
        print(x_train_stock_CKIP_nonEng.index[i])

In [44]:
x_train_stock_CKIP_nonEng = x_train_stock_CKIP_nonEng.values.tolist()
x_test_stock_CKIP_nonEng = x_test_stock_CKIP_nonEng.values.tolist()

### 設定 SVM 參數

In [26]:
SVM_model = SVC(kernel = "linear", C = 1.0, probability=True)

### SVM Model 函式

In [27]:
def tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding):
    
    t0 = time()
    SVM_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(SVM_model.predict(x_test))
    
    print(tokens_name + " predict " + predict_stocks_name + "(SVM Linear + " + embedding + ")")
    print(metrics.classification_report(excepted_results,predicted_results))


## SVD vector

### covid 板 jieba 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [27]:
tokens_predict_stocks(x_train_covid_token, y_train_mask, x_test_covid_token, y_test_mask, "Covid jieba token", "mask stock")

done in 655.555s.
Covid jieba token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.38      0.47      0.42       557
           0       0.18      0.01      0.01       324
           1       0.42      0.55      0.48       622

    accuracy                           0.40      1503
   macro avg       0.33      0.34      0.30      1503
weighted avg       0.36      0.40      0.36      1503



#### 預測檢測試劑類股漲跌效果

In [75]:
tokens_predict_stocks(x_train_covid_token, y_train_testKits, x_test_covid_token, y_test_testKits, "Covid jieba token", "testKits stock")

done in 557.543s.
Covid jieba token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.30      0.03      0.06       416
           0       0.43      0.92      0.59       636
           1       0.33      0.07      0.11       451

    accuracy                           0.42      1503
   macro avg       0.35      0.34      0.25      1503
weighted avg       0.36      0.42      0.30      1503



#### 預測疫苗類股漲跌效果

In [76]:
tokens_predict_stocks(x_train_covid_token, y_train_vaccine, x_test_covid_token, y_test_vaccine, "Covid jieba token", "vaccine stock")

done in 1103.772s.
Covid jieba token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.50      0.03      0.06       323
           0       0.53      0.98      0.69       787
           1       0.64      0.05      0.09       393

    accuracy                           0.53      1503
   macro avg       0.56      0.35      0.28      1503
weighted avg       0.55      0.53      0.40      1503



### covid 板 ckip 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [45]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_mask, x_test_covid_CKIPtoken, y_test_mask, "Covid CKIP token", "mask stock")

done in 629.538s.
Covid CKIP token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.39      0.50      0.44       557
           0       0.27      0.01      0.02       324
           1       0.42      0.52      0.46       622

    accuracy                           0.40      1503
   macro avg       0.36      0.34      0.31      1503
weighted avg       0.38      0.40      0.36      1503



#### 預測檢測試劑類股漲跌效果

In [77]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_testKits, x_test_covid_CKIPtoken, y_test_testKits, "Covid CKIP token", "testKits stock")

done in 581.769s.
Covid CKIP token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.34      0.04      0.07       416
           0       0.43      0.92      0.59       636
           1       0.42      0.09      0.15       451

    accuracy                           0.43      1503
   macro avg       0.40      0.35      0.27      1503
weighted avg       0.40      0.43      0.32      1503



#### 預測疫苗類股漲跌效果

In [72]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_vaccine, x_test_covid_CKIPtoken, y_test_vaccine, "Covid CKIP token", "vaccine stock")

done in 1142.909s.
Covid CKIP token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.52      0.03      0.06       323
           0       0.53      0.98      0.69       787
           1       0.61      0.05      0.09       393

    accuracy                           0.53      1503
   macro avg       0.55      0.36      0.28      1503
weighted avg       0.55      0.53      0.40      1503



### covid 板 ckip non Eng 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [19]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_mask, x_test_covid_CKIP_nonEng, y_test_mask, "Covid CKIP nonEng token", "mask stock")

done in 600.380s.
Covid CKIP nonEng token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.39      0.50      0.44       557
           0       0.21      0.01      0.02       324
           1       0.43      0.53      0.48       622

    accuracy                           0.41      1503
   macro avg       0.34      0.35      0.31      1503
weighted avg       0.37      0.41      0.36      1503



#### 預測檢測試劑類股漲跌效果

In [20]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_testKits, x_test_covid_CKIP_nonEng, y_test_testKits, "Covid CKIP nonEng token", "testKits stock")

done in 541.022s.
Covid CKIP nonEng token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.29      0.04      0.06       416
           0       0.43      0.91      0.58       636
           1       0.34      0.09      0.14       451

    accuracy                           0.42      1503
   macro avg       0.36      0.34      0.26      1503
weighted avg       0.37      0.42      0.31      1503



#### 預測疫苗類股漲跌效果

In [21]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_vaccine, x_test_covid_CKIP_nonEng, y_test_vaccine, "Covid CKIP nonEng token", "vaccine stock")

done in 1025.492s.
Covid CKIP nonEng token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.62      0.03      0.06       323
           0       0.53      0.99      0.69       787
           1       0.61      0.04      0.08       393

    accuracy                           0.53      1503
   macro avg       0.59      0.35      0.28      1503
weighted avg       0.57      0.53      0.40      1503



### stock 板 jieba 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [48]:
tokens_predict_stocks(x_train_stock_token, y_train_mask, x_test_stock_token, y_test_mask, "Stock jieba token", "mask stock")

done in 915.240s.
Stock jieba token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.38      0.31      0.34       695
           0       0.29      0.01      0.02       458
           1       0.41      0.70      0.51       793

    accuracy                           0.40      1946
   macro avg       0.36      0.34      0.29      1946
weighted avg       0.37      0.40      0.34      1946



#### 預測檢測試劑類股漲跌效果

In [49]:
tokens_predict_stocks(x_train_stock_token, y_train_testKits, x_test_stock_token, y_test_testKits, "Stock jieba token", "testKits stock")

done in 653.578s.
Stock jieba token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.43      0.07      0.12       525
           0       0.45      0.94      0.61       855
           1       0.28      0.03      0.06       566

    accuracy                           0.44      1946
   macro avg       0.38      0.35      0.26      1946
weighted avg       0.39      0.44      0.32      1946



#### 預測疫苗類股漲跌效果

In [50]:
tokens_predict_stocks(x_train_stock_token, y_train_vaccine, x_test_stock_token, y_test_vaccine, "Stock jieba token", "vaccine stock")

done in 2636.979s.
Stock jieba token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.12      0.00      0.01       428
           0       0.56      0.99      0.72      1082
           1       0.75      0.01      0.01       436

    accuracy                           0.56      1946
   macro avg       0.48      0.34      0.25      1946
weighted avg       0.51      0.56      0.40      1946



### stock 板 ckip 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [54]:
tokens_predict_stocks(x_train_stock_CKIPtoken, y_train_mask, x_test_stock_CKIPtoken, y_test_mask, "Stock CKIP token", "mask stock")

done in 1118.763s.
Stock CKIP token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.38      0.31      0.34       695
           0       0.14      0.00      0.00       458
           1       0.41      0.71      0.52       793

    accuracy                           0.40      1946
   macro avg       0.31      0.34      0.29      1946
weighted avg       0.34      0.40      0.33      1946



#### 預測檢測試劑類股漲跌效果

In [55]:
tokens_predict_stocks(x_train_stock_CKIPtoken, y_train_testKits, x_test_stock_CKIPtoken, y_test_testKits, "Stock CKIP token", "testKits stock")

done in 722.631s.
Stock CKIP token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.33      0.06      0.10       525
           0       0.44      0.93      0.60       855
           1       0.23      0.02      0.04       566

    accuracy                           0.43      1946
   macro avg       0.34      0.34      0.25      1946
weighted avg       0.35      0.43      0.30      1946



#### 預測疫苗類股漲跌效果

In [56]:
tokens_predict_stocks(x_train_stock_token, y_train_vaccine, x_test_stock_token, y_test_vaccine, "Stock jieba token", "vaccine stock")

done in 2643.753s.
Stock jieba token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.12      0.00      0.01       428
           0       0.56      0.99      0.72      1082
           1       0.75      0.01      0.01       436

    accuracy                           0.56      1946
   macro avg       0.48      0.34      0.25      1946
weighted avg       0.51      0.56      0.40      1946



### stock 板 ckip non Eng 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [64]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_mask, x_test_stock_CKIP_nonEng, y_test_mask, "Stock CKIP nonEng token", "mask stock")

done in 1031.173s.
Stock CKIP nonEng token predict mask stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.38      0.33      0.35       695
           0       0.12      0.00      0.00       458
           1       0.41      0.70      0.52       793

    accuracy                           0.40      1946
   macro avg       0.31      0.34      0.29      1946
weighted avg       0.33      0.40      0.34      1946



#### 預測檢測試劑類股漲跌效果

In [65]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_testKits, x_test_stock_CKIP_nonEng, y_test_testKits, "Stock CKIP nonEng token", "testKits stock")

done in 666.666s.
Stock CKIP nonEng token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.35      0.06      0.11       525
           0       0.44      0.92      0.60       855
           1       0.29      0.04      0.07       566

    accuracy                           0.43      1946
   macro avg       0.36      0.34      0.26      1946
weighted avg       0.38      0.43      0.31      1946



#### 預測疫苗類股漲跌效果

In [66]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_vaccine, x_test_stock_CKIP_nonEng, y_test_vaccine, "Stock CKIP nonEng token", "vaccine stock")

done in 2547.866s.
Stock CKIP nonEng token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.10      0.00      0.00       428
           0       0.56      0.99      0.72      1082
           1       0.44      0.01      0.02       436

    accuracy                           0.56      1946
   macro avg       0.37      0.34      0.25      1946
weighted avg       0.43      0.56      0.40      1946



## word2vec

### covid 板 jieba 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [91]:
tokens_predict_stocks(x_train_covid_token, y_train_mask, x_test_covid_token, y_test_mask, "Covid jieba token", "mask stock", "word2vec")

done in 1586.404s.
Covid jieba token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.40      0.49      0.44       557
           0       0.00      0.00      0.00       324
           1       0.44      0.57      0.49       622

    accuracy                           0.42      1503
   macro avg       0.28      0.35      0.31      1503
weighted avg       0.33      0.42      0.37      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [96]:
tokens_predict_stocks(x_train_covid_token, y_train_testKits, x_test_covid_token, y_test_testKits, "Covid jieba token", "testKits stock", "word2vec")

done in 711.945s.
Covid jieba token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.40      0.00      0.01       416
           0       0.42      0.98      0.59       636
           1       0.33      0.02      0.04       451

    accuracy                           0.42      1503
   macro avg       0.39      0.33      0.21      1503
weighted avg       0.39      0.42      0.26      1503



#### 預測疫苗類股漲跌效果

In [97]:
tokens_predict_stocks(x_train_covid_token, y_train_vaccine, x_test_covid_token, y_test_vaccine, "Covid jieba token", "vaccine stock", "word2vec")

done in 7563.613s.
Covid jieba token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       323
           0       0.52      1.00      0.69       787
           1       0.00      0.00      0.00       393

    accuracy                           0.52      1503
   macro avg       0.17      0.33      0.23      1503
weighted avg       0.27      0.52      0.36      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### covid 板 ckip 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [98]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_mask, x_test_covid_CKIPtoken, y_test_mask, "Covid CKIP token", "mask stock", "word2vec")

done in 902.291s.
Covid CKIP token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.42      0.55      0.47       557
           0       0.00      0.00      0.00       324
           1       0.44      0.55      0.49       622

    accuracy                           0.43      1503
   macro avg       0.29      0.36      0.32      1503
weighted avg       0.34      0.43      0.38      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [99]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_testKits, x_test_covid_CKIPtoken, y_test_testKits, "Covid CKIP token", "testKits stock", "word2vec")

done in 725.022s.
Covid CKIP token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.50      0.00      0.01       416
           0       0.42      0.98      0.59       636
           1       0.23      0.01      0.03       451

    accuracy                           0.42      1503
   macro avg       0.38      0.33      0.21      1503
weighted avg       0.39      0.42      0.26      1503



#### 預測疫苗類股漲跌效果

In [31]:
tokens_predict_stocks(x_train_covid_CKIPtoken, y_train_vaccine, x_test_covid_CKIPtoken, y_test_vaccine, "Covid CKIP token", "vaccine stock", "word2vec")

done in 4181.833s.
Covid CKIP token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       323
           0       0.52      1.00      0.69       787
           1       0.00      0.00      0.00       393

    accuracy                           0.52      1503
   macro avg       0.17      0.33      0.23      1503
weighted avg       0.27      0.52      0.36      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### covid 板 ckip non Eng 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [28]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_mask, x_test_covid_CKIP_nonEng, y_test_mask, "Covid CKIP nonEng token", "mask stock", "word2vec")

done in 955.370s.
Covid CKIP nonEng token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.41      0.52      0.46       557
           0       0.00      0.00      0.00       324
           1       0.43      0.55      0.48       622

    accuracy                           0.42      1503
   macro avg       0.28      0.36      0.31      1503
weighted avg       0.33      0.42      0.37      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [29]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_testKits, x_test_covid_CKIP_nonEng, y_test_testKits, "Covid CKIP nonEng token", "testKits stock", "word2vec")

done in 760.348s.
Covid CKIP nonEng token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       416
           0       0.42      0.97      0.59       636
           1       0.32      0.03      0.06       451

    accuracy                           0.42      1503
   macro avg       0.25      0.33      0.22      1503
weighted avg       0.28      0.42      0.27      1503



#### 預測疫苗類股漲跌效果

In [30]:
tokens_predict_stocks(x_train_covid_CKIP_nonEng, y_train_vaccine, x_test_covid_CKIP_nonEng, y_test_vaccine, "Covid CKIP nonEng token", "vaccine stock", "word2vec")

done in 5037.822s.
Covid CKIP nonEng token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       323
           0       0.52      1.00      0.69       787
           1       0.00      0.00      0.00       393

    accuracy                           0.52      1503
   macro avg       0.17      0.33      0.23      1503
weighted avg       0.27      0.52      0.36      1503



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### stock 板 jieba 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [45]:
tokens_predict_stocks(x_train_stock_token, y_train_mask, x_test_stock_token, y_test_mask, "Stock jieba token", "mask stock", "word2vec")

done in 3843.164s.
Stock jieba token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.36      0.30      0.33       695
           0       0.00      0.00      0.00       458
           1       0.41      0.70      0.51       793

    accuracy                           0.39      1946
   macro avg       0.25      0.33      0.28      1946
weighted avg       0.29      0.39      0.33      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [46]:
tokens_predict_stocks(x_train_stock_token, y_train_testKits, x_test_stock_token, y_test_testKits, "Stock jieba token", "testKits stock", "word2vec")

done in 1164.222s.
Stock jieba token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       525
           0       0.44      1.00      0.61       855
           1       0.00      0.00      0.00       566

    accuracy                           0.44      1946
   macro avg       0.15      0.33      0.20      1946
weighted avg       0.19      0.44      0.27      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測疫苗類股漲跌效果

In [47]:
tokens_predict_stocks(x_train_stock_token, y_train_vaccine, x_test_stock_token, y_test_vaccine, "Stock jieba token", "vaccine stock", "word2vec")

done in 10492.473s.
Stock jieba token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       428
           0       0.56      1.00      0.71      1082
           1       0.00      0.00      0.00       436

    accuracy                           0.56      1946
   macro avg       0.19      0.33      0.24      1946
weighted avg       0.31      0.56      0.40      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### stock 板 ckip 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [48]:
tokens_predict_stocks(x_train_stock_CKIPtoken, y_train_mask, x_test_stock_CKIPtoken, y_test_mask, "Stock CKIP token", "mask stock", "word2vec")

done in 2900.940s.
Stock CKIP token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.35      0.31      0.33       695
           0       0.00      0.00      0.00       458
           1       0.40      0.67      0.50       793

    accuracy                           0.38      1946
   macro avg       0.25      0.33      0.28      1946
weighted avg       0.29      0.38      0.32      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [49]:
tokens_predict_stocks(x_train_stock_CKIPtoken, y_train_testKits, x_test_stock_CKIPtoken, y_test_testKits, "Stock CKIP token", "testKits stock", "word2vec")

done in 708.970s.
Stock CKIP token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       525
           0       0.44      0.99      0.61       855
           1       0.40      0.00      0.01       566

    accuracy                           0.44      1946
   macro avg       0.28      0.33      0.21      1946
weighted avg       0.31      0.44      0.27      1946



#### 預測疫苗類股漲跌效果

In [50]:
tokens_predict_stocks(x_train_stock_CKIPtoken, y_train_vaccine, x_test_stock_CKIPtoken, y_test_vaccine, "Stock CKIP token", "vaccine stock", "word2vec")

done in 10948.482s.
Stock CKIP token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       428
           0       0.56      1.00      0.71      1082
           1       0.00      0.00      0.00       436

    accuracy                           0.56      1946
   macro avg       0.19      0.33      0.24      1946
weighted avg       0.31      0.56      0.40      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### stock 板 ckip non Eng 斷詞（加入情緒詞）

#### 預測口罩類股漲跌效果

In [51]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_mask, x_test_stock_CKIP_nonEng, y_test_mask, "Stock CKIP nonEng token", "mask stock", "word2vec")

done in 2580.877s.
Stock CKIP nonEng token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.36      0.30      0.33       695
           0       0.00      0.00      0.00       458
           1       0.41      0.70      0.52       793

    accuracy                           0.39      1946
   macro avg       0.26      0.34      0.28      1946
weighted avg       0.30      0.39      0.33      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測檢測試劑類股漲跌效果

In [52]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_testKits, x_test_stock_CKIP_nonEng, y_test_testKits, "Stock CKIP nonEng token", "testKits stock", "word2vec")

done in 757.620s.
Stock CKIP nonEng token predict testKits stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       525
           0       0.44      1.00      0.61       855
           1       0.00      0.00      0.00       566

    accuracy                           0.44      1946
   macro avg       0.15      0.33      0.20      1946
weighted avg       0.19      0.44      0.27      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 預測疫苗類股漲跌效果

In [53]:
tokens_predict_stocks(x_train_stock_CKIP_nonEng, y_train_vaccine, x_test_stock_CKIP_nonEng, y_test_vaccine, "Stock CKIP nonEng token", "vaccine stock", "word2vec")

done in 10754.100s.
Stock CKIP nonEng token predict vaccine stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       428
           0       0.56      1.00      0.71      1082
           1       0.00      0.00      0.00       436

    accuracy                           0.56      1946
   macro avg       0.19      0.33      0.24      1946
weighted avg       0.31      0.56      0.40      1946



C:\Users\molly\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
print("well done! YUO MADE IT!")

well done! YUO MADE IT!


#### 預測口罩類股漲跌效果

In [178]:
t0 = time()
SVM_model.fit(x_train, y_train_mask)
print("done in %0.3fs." % (time() - t0))

done in 657.773s.


In [176]:
predicted_results = []
excepted_results = []

excepted_results.extend(y_test_mask)
predicted_results.extend(SVM_model.predict(x_test))

[-1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 0, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1

In [179]:
print("Covid_token predict mask stock (SVM Linear + SVD vector)")
print(metrics.classification_report(excepted_results,predicted_results))

Covid_token predict mask stock (SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.38      0.47      0.42       557
           0       0.18      0.01      0.01       324
           1       0.42      0.55      0.48       622

    accuracy                           0.40      1503
   macro avg       0.33      0.34      0.30      1503
weighted avg       0.36      0.40      0.36      1503

